# Feature Engineering for Bimbo Competition

Note: Please refer to Data Explore for more info on the Data 

Data fields

    Semana — Week number (From Thursday to Wednesday)
    Agencia_ID — Sales Depot ID
    Canal_ID — Sales Channel ID
    Ruta_SAK — Route ID (Several routes = Sales Depot)
    Cliente_ID — Client ID
    NombreCliente — Client name
    Producto_ID — Product ID
    NombreProducto — Product Name
    Venta_uni_hoy — Sales unit this week (integer)          <----- FUTURE
    Venta_hoy — Sales this week (unit: pesos)               <----- FUTURE
    Dev_uni_proxima — Returns unit next week (integer)      <----- FUTURE
    Dev_proxima — Returns next week (unit: pesos)           <----- FUTURE
    Demanda_uni_equil — Adjusted Demand (integer) (This is the target you will predict)

Things to note:

    There may be products in the test set that don't exist in the train set. This is the expected behavior of inventory data, since there are new products being sold all the time. Your model should be able to accommodate this.
    There are duplicate Cliente_ID's in cliente_tabla, which means one Cliente_ID may have multiple NombreCliente that are very similar. This is due to the NombreCliente being noisy and not standardized in the raw data, so it is up to you to decide how to clean up and use this information. 
    The adjusted demand (Demanda_uni_equil) is always >= 0 since demand should be either 0 or a positive value. The reason that Venta_uni_hoy - Dev_uni_proxima sometimes has negative values is that the returns records sometimes carry over a few weeks.
    
    

## Features 

    #1 Categorical Values (IDs): Agencia, Canal, Ruta, Cliente,Producto
        I should do one hot encoding for all the categorical values. But there are just too many
    #2 Location Features (Categorical): ZipCode, from the towns table we extract the ZipCode
    
    #3 Numerical Values: Last Demand per Client-Product and Route-Product pairs 
        These features differs from lagged features because I don't care on which week was the previous order
        
    #4 Numerical Values: Means and Medians of the Demand
        Mean/Median 1 : Global mean/median
        Mean/Median 2 : Client-Product pair mean/median
        Mean/Median 3 : Route-Client-Product pair mean/median
        Mean/Median 4 : Route-Product pair mean/median
        Mean/Median 5 : ZipCode-Product pair mean/median
        Mean/Median 6 : Client mean/median
        Mean/Median 7 : Route-Client pair mean/median
        Mean/Median 8 : ZipCode-Product pair mean/median
        Mean/Median 9 : Route mean/median
        
    #5 Week Counter (from 0 to 3) to identify monthly patterns

In [ ]:
use_w10 = True

### Imports

In [2]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import xgboost as xgb

C:\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
import pandas as pd
import numpy as np
import re 
import os
import time
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.cross_validation import LabelKFold
from sklearn.grid_search import GridSearchCV
import seaborn as sns
%matplotlib inline

## Categorical Features (ID's) and Location Features

In [4]:

def load_towns_table(folder = ''):
    
    filename = "town_state.csv"
    filepath = os.path.join(folder,filename)    
        
    towns = pd.read_csv(filepath)
    L = lambda x: map(int, re.findall('\d+', x))[0]
    towns['ZipCode'] = towns.Town.apply(L) 
    towns['ZipCode'] = np.uint16(towns['ZipCode'])
        
    return towns[['Agencia_ID','ZipCode']]

def load_clients_table(folder = ''):
        
    filename = "cliente_tabla.csv"
    filepath = os.path.join(folder,filename)        
    
    clients = pd.read_csv(filepath)
    clients = clients.drop_duplicates('Cliente_ID') # Remove duplicates rows
    
    return clients

def load_products_table(folder = ''):
    
    filename = "producto_tabla.csv"
    filepath = os.path.join(folder,filename)    
    
    products = pd.read_csv(filepath)
    
    return products

def load_train_table(folder = ''):
    
    filename = "train.csv"
    #filename = "train_last.csv"
    filepath = os.path.join(folder,filename)        
    
    df_train = pd.read_csv(filepath,
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'Venta_hoy':'float32',
                                     'Venta_uni_hoy': 'int32',
                                     'Dev_uni_proxima':'int32',
                                     'Dev_proxima':'float32',
                                     'Demanda_uni_equil':'int32'})
    
    return df_train

def load_last_table(folder = ''):
    
    
    filename = "train_last.csv"
    filepath = os.path.join(folder,filename)        
    
    df_train = pd.read_csv(filepath,
                           usecols = ['Semana', 'Producto_ID', 'Cliente_ID', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'ZipCode',
                                      'Last_per_Agencia_ID', 'Last_per_Canal_ID','Last_per_Ruta_SAK','Last_per_Cliente_ID',
                                      'Last_per_ZipCode','Demanda_uni_equil'],
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'ZipCode':'int16',                                     
                                     'Last_per_Agencia_ID':'float32',
                                     'Last_per_Canal_ID':'float32',
                                     'Last_per_Ruta_SAK':'float32',
                                     'Last_per_Cliente_ID':'float32',
                                     'Last_per_ZipCode':'float32',
                                     'Demanda_uni_equil':'int32'})
    return df_train



def load_test_table(folder = ''):
    
    filename = "test.csv"
    filepath = os.path.join(folder,filename)  
    
    df_test = pd.read_csv(filepath,
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32'})

    
    towns = load_towns_table(folder)
    
    
    df_test = df_test.reset_index(drop=True).set_index('Agencia_ID').join(towns.reset_index(drop=True).set_index('Agencia_ID'))
    return df_test.reset_index()

def load_bimbo_data(folder = ''):
    
    towns = load_towns_table(folder)
    clients = load_clients_table(folder)
    products = load_products_table(folder)
    df_train = load_train_table(folder)   
        
    #giant_df = df_train.merge(clients, on = 'Cliente_ID')
    #giant_df = giant_df.merge(products, on='Producto_ID')
    giant_df = df_train.merge(towns, on ='Agencia_ID', how='left')
    
    return giant_df

def merge_train_test(df_train, df_test):
    #format columns
    add_to_train =  np.setdiff1d(df_test.columns, df_train.columns)
    add_to_test =  np.setdiff1d(df_train.columns, df_test.columns)
    
    print "Adding columns to train table" ,add_to_train
    for col in add_to_train:
        df_train[col] = np.float32(np.nan) 
    print "Adding columns to test table" ,add_to_test
    for col in add_to_test:
        df_test[col] = np.float32(np.nan)
    
    df = pd.concat([df_train,df_test])
    df = df.fillna(0)        
    
    return df 
    

In [5]:
%time df_train = load_bimbo_data()
%time df_test = load_test_table()

%time df = merge_train_test(df_train,df_test)
del df_test
del df_train

display(df.head())
#Let's check the dtypes for memory 
print df.dtypes


Wall time: 1min 32s
Wall time: 8.49 s
Adding columns to train table ['id']
Adding columns to test table ['Demanda_uni_equil' 'Dev_proxima' 'Dev_uni_proxima' 'Venta_hoy'
 'Venta_uni_hoy']
Wall time: 21.3 s


,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,ZipCode,id
0,1110,7,15766,3.0,0.0,0.0,1212,3301,3,25.139999,3.0,2008,0.0
1,1110,7,15766,4.0,0.0,0.0,1216,3301,3,33.520000,4.0,2008,0.0
2,1110,7,15766,4.0,0.0,0.0,1238,3301,3,39.320000,4.0,2008,0.0
3,1110,7,15766,4.0,0.0,0.0,1240,3301,3,33.520000,4.0,2008,0.0
4,1110,7,15766,3.0,0.0,0.0,1242,3301,3,22.920000,3.0,2008,0.0


Agencia_ID             int64
Canal_ID                int8
Cliente_ID             int32
Demanda_uni_equil    float64
Dev_proxima          float64
Dev_uni_proxima      float64
Producto_ID            int32
Ruta_SAK               int32
Semana                  int8
Venta_hoy            float64
Venta_uni_hoy        float64
ZipCode               uint16
id                   float64
dtype: object


In [6]:
# Memory Saving Tips 

df.Demanda_uni_equil = np.int32(df.Demanda_uni_equil)
df.ZipCode = np.int16(df.ZipCode)
df.Agencia_ID = np.int32(df.Agencia_ID)
df.id = np.int32(df.id)

df.dtypes
# I choose to ignore returns and Venta 

Agencia_ID             int32
Canal_ID                int8
Cliente_ID             int32
Demanda_uni_equil      int32
Dev_proxima          float64
Dev_uni_proxima      float64
Producto_ID            int32
Ruta_SAK               int32
Semana                  int8
Venta_hoy            float64
Venta_uni_hoy        float64
ZipCode                int16
id                     int32
dtype: object

## For Better Results 

Apply Log1p to Demanda_uni_proxima

In [7]:
df.Demanda_uni_equil = df.Demanda_uni_equil.apply(lambda x: np.log1p(x))

## Numerical Features 

### Last Feature

For the Last feature I'm going to cluster per Cliente-Producto pair and the Route-Product pair

In [8]:
def add_Last(df ,clusters = ['Cliente_ID']):
    
    
    validclusters = np.intersect1d(clusters, df.columns)
    not_valid_clusters = np.setdiff1d(clusters, df.columns)
    
    for non in not_valid_clusters:
        
        print "Cluster ", non, " not found in the dataset"            
    
    drop_columns = ['Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima']
    try:
        df = df.drop(drop_columns, 1) # Try to drop the columns if the dataset has them
    except:
        df = df # no drop 
    
    for cluster in validclusters:
        
        print 'Working on cluster:', cluster                                                    
        
        target_col = 'Last_per_'+cluster
        group = [cluster,'Producto_ID','Semana']
                    
        try:
            df = df.drop(target_col,1) # If the target exist then remove it
        except:
            df = df 
        
        # First Compute the Mean by grouping the Cluster-Producto per Week Demand 
        # Then shift by Product_ID - we want to treat each products independent 
        # Lastly reset the index because we don't wanna change the dataset
        shifted_df = ( df.groupby(group).agg({'Demanda_uni_equil': np.mean})['Demanda_uni_equil'] \
                                         .groupby(level=1).shift(1) \
                                        .reset_index())
        
        shifted_df = shifted_df.fillna(0)   # Fill NaN with 0 
        shifted_df['Demanda_uni_equil'] = np.float32(shifted_df['Demanda_uni_equil']) #reset index resets the dtypes
        shifted_df.rename(columns={'Demanda_uni_equil': 'Last_per_'+cluster }, inplace=True) # rename cols        
        df = df.merge(shifted_df, on=group, how='left')
        print df.dtypes
        
        #Note to self: break the line using \                               
        
    return df

In [9]:
df = add_Last(df,['Cliente_ID','Ruta_SAK'])

Working on cluster: Cliente_ID
Agencia_ID               int32
Canal_ID                  int8
Cliente_ID               int32
Demanda_uni_equil      float64
Producto_ID              int32
Ruta_SAK                 int32
Semana                    int8
ZipCode                  int16
id                       int32
Last_per_Cliente_ID    float32
dtype: object
Working on cluster: Ruta_SAK
Agencia_ID               int32
Canal_ID                  int8
Cliente_ID               int32
Demanda_uni_equil      float64
Producto_ID              int32
Ruta_SAK                 int32
Semana                    int8
ZipCode                  int16
id                       int32
Last_per_Cliente_ID    float32
Last_per_Ruta_SAK      float32
dtype: object


In [10]:
df.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0


### Means and Medians

I'm only computing Mean 2/3/4/7/6 to save memory 

In [11]:
def add_means_medians(df):
    groups_dict = { '2' :['Cliente_ID','Producto_ID'], '3': ['Ruta_SAK','Cliente_ID','Producto_ID'] ,
               '4' :['Ruta_SAK','Producto_ID'],'7':['Ruta_SAK','Cliente_ID'], 
               '6':['Cliente_ID']}
    
    
    for key,value in groups_dict.items():
        print "Working on :" ,key, value
        print "df cols: ", df.columns
        start = time.time() 
                                        
        df_mean = df[df.Semana < 9].groupby(value, as_index=False).agg({'Demanda_uni_equil': np.mean})
        df_mean.rename(columns={'Demanda_uni_equil': 'Mean'+key }, inplace=True) # rename cols                
        df_mean['Mean'+key] = np.float32(df_mean['Mean'+key])
        df = df.merge(df_mean, on=value,how='left')
        
        print "df_mean cols: ", df_mean.columns
        del df_mean
        
        
        df_median = df[df.Semana < 9].groupby(value, as_index=False).agg({'Demanda_uni_equil': np.median})
        df_median.rename(columns={'Demanda_uni_equil': 'Median'+key }, inplace=True) # rename cols                
        df_median['Median'+key] = np.float32(df_median['Median'+key])
        df = df.merge(df_median, on=value, how='left')
                
        print "df_median cols: ",df_median.columns
        del df_median
        print("total time taken this loop: ", time.time() - start)
            
    print "Final cols:", df.columns
    return df            

In [12]:
# Compute the Means and medians
#df = add_means_medians(df)
display(df.head())

# update: This feature really hurts the model

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0


In [13]:
df.dtypes

Agencia_ID               int32
Canal_ID                  int8
Cliente_ID               int32
Demanda_uni_equil      float64
Producto_ID              int32
Ruta_SAK                 int32
Semana                    int8
ZipCode                  int16
id                       int32
Last_per_Cliente_ID    float32
Last_per_Ruta_SAK      float32
dtype: object

### Week Counter 

In [14]:
# Create the Week Counter
df['week_ct'] = df['Semana'].apply(lambda x: x%4)

In [15]:
df.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0,3
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0,3
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0,3
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0,3
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0,3


In [16]:
#df.to_csv('jorge_dataset.csv',index=False) 
# 16GB dataset Woot!

# Team Datos Feature Engineering

## Pablo's Features 

    * Lagged Features - Demand per Client-Product pair (up to 4) 
    * Sum of Prior Demands
    * Product Brand 
    * Qty of Routes that Serve a Client

### Feature1: Lag Features - Demand per client-product pair for prior weeks


In [17]:
semana_client_prod_mean = df.groupby(['Semana','Cliente_ID','Producto_ID'],as_index=False).agg({'Demanda_uni_equil': 'mean'})

#here we add the number of lags we want
lag=4
target_col = 'Demanda_uni_equil'
for i in range(1,lag+1):
    semana_client_prod_mean['Semana'] += 1
    semana_client_prod_mean.rename(columns={target_col: 'Log_Target_mean_lag%d' %(i)}, inplace=True)
    df = pd.merge(df,semana_client_prod_mean, how = 'left', on = ['Semana','Cliente_ID','Producto_ID'])
    df['Log_Target_mean_lag%d' %(i)].fillna(0, inplace=True) # we replace the client-product log mean NaN/Not found on the week before with ZERO
    target_col = 'Log_Target_mean_lag%d' %(i)
    #data = data[data.Semana != i+2] # here we delete the week rows we dont have lags for


In [18]:
df.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0,3,0.0,0.0,0.0,0.0
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0


###  Feature 2:  Calculates de sum of prior weeks Log mean Demands

In [19]:
#We want to sum the lags up until week 9, this means that we need to sum lag2 and up.
df['Lags_sum'] = 0
for i in range(1,lag+1):
    df['Lags_sum'] += df['Log_Target_mean_lag%d' %(i)]

In [20]:
df.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0,3,0.0,0.0,0.0,0.0,0.0
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0,0.0
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0,0.0
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0,0.0
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0,0.0


### Feature 4: Create a broad category of Brand of item (brand hypothesis)


In [21]:
products  =  load_products_table()

products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
#products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$')
products.head()

,Producto_ID,NombreProducto,brand
0,0,NO IDENTIFICADO 0,IDENTIFICADO
1,9,Capuccino Moka 750g NES 9,NES
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,BIM
3,53,Burritos Sincro 170g CU LON 53,LON
4,72,Div Tira Mini Doradita 4p 45g TR 72,TR


In [22]:
products_id_brand  = products[['Producto_ID', 'brand']].copy()
df = pd.merge(df, products_id_brand, on='Producto_ID', how='left')

### Feature 6: Create a category of Size of store based on Number of Agencies and Routes and Sales Channels that serve the store

In [23]:
#Determine pivot table
Rutas_per_store = df.pivot_table(values=["Ruta_SAK"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [24]:
Rutas_per_store.rename(columns={'Ruta_SAK': 'Qty_Ruta_SAK'}, inplace=True)

In [25]:
#Mergin Routa_Sak's per client to data df
df = pd.merge(df,Rutas_per_store,right_index=True, left_on='Cliente_ID')
df.tail()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK
81178661,25759,1,4745721,0.0,43040,5504,11,2575,6667523,1.098612,0.000000,3,0.0,0.0,0.0,0.0,0.0,TR,1
81176435,25759,1,8021338,0.0,43307,2814,11,2575,5982320,0.000000,0.000000,3,0.0,0.0,0.0,0.0,0.0,MLA,1
81177618,25759,1,8021338,0.0,37361,2814,11,2575,6356826,1.609438,0.000000,3,0.0,0.0,0.0,0.0,0.0,MLA,1
81179389,25759,1,8021338,0.0,30532,2814,11,2575,6890213,0.000000,0.000000,3,0.0,0.0,0.0,0.0,0.0,MLA,1
81177158,25759,1,2296849,0.0,31586,2131,10,2575,6217504,0.693147,1.284725,2,0.0,0.0,0.0,0.0,0.0,MLA,1


In [26]:
#Binning:
def binning(col, cut_points, labels=None):
    #Define min and max values:
    minval = col.min()
    maxval = col.max()

    #create list by adding min and max to cut_points
    break_points = [minval] + cut_points + [maxval]

    #if no labels provided, use default labels 0 ... (n-1)
    if not labels:
        labels = range(len(cut_points)+1)

    #Binning using cut function of pandas
    colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
    return colBin

#Binning Qty_Ruta_SAK:
cut_points = [2,4,10]
labels = ["low","medium","high","very high"]
df["Qty_Ruta_SAK_Bin"] = binning(df["Qty_Ruta_SAK"], cut_points, labels)
print (pd.value_counts(df["Qty_Ruta_SAK_Bin"], sort=False))

low          24879666
medium       28933020
high         27203419
very high      163610
dtype: int64


In [27]:
#We don't need Qty_Ruta_Sak anymore
df.drop(['Qty_Ruta_SAK'],axis=1,inplace=True)

In [28]:
df.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK_Bin
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0,3,0.0,0.0,0.0,0.0,0.0,BIM,low
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0,0.0,BIM,low
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0,0.0,BIM,low
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0,0.0,BIM,low
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0,0.0,BIM,low


### Numerical and One-Hot Coding of Categorical variables
Since scikit-learn accepts only numerical variables, so i have to convert all categories of nominal variables into numeric types.

Lets start with coding all low cardinality object/nominal categorical variables (brand, Qty_Ruta_SAK_Bin)  as numeric using ‘LabelEncoder’ from sklearn’s preprocessing module.

In [29]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

var_mod = ['brand', 'Qty_Ruta_SAK_Bin']
for i in var_mod:
    df[i] = le.fit_transform(df[i])

In [30]:
df.head()

,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,Last_per_Cliente_ID,Last_per_Ruta_SAK,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK_Bin
0,1110,7,15766,1.386294,1212,3301,3,2008,0,1.609438,0.0,3,0.0,0.0,0.0,0.0,0.0,2,1
1,1110,7,15766,1.609438,1216,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0,0.0,2,1
2,1110,7,15766,1.609438,1238,3301,3,2008,0,0.000000,0.0,3,0.0,0.0,0.0,0.0,0.0,2,1
3,1110,7,15766,1.609438,1240,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0,0.0,2,1
4,1110,7,15766,1.386294,1242,3301,3,2008,0,1.098612,0.0,3,0.0,0.0,0.0,0.0,0.0,2,1


## Luisa Features 

    * Number of Orders in the current Week 

In [31]:
# for every client, get the number of products ordered in the current week
df_num_prods = df.groupby(['Semana','Cliente_ID'])['Producto_ID'].apply(lambda x: len(x)).reset_index()
df_num_prods.rename(columns={df_num_prods.columns[-1]: 'num_prod'}, inplace=True)


# for every client, get the number of unique products ordered in the current week
df_num_prods_unique = df.groupby(['Semana','Cliente_ID'])['Producto_ID'].apply(lambda x: len(x.unique())).reset_index()
df_num_prods_unique.rename(columns={df_num_prods_unique.columns[-1]: 'num_prod_uni'}, inplace=True)

# merge to the dataset
df = df.merge(df_num_prods,on=['Semana','Cliente_ID'], how='left')
df = df.merge(df_num_prods_unique, on=['Semana','Cliente_ID'], how='left')


In [32]:
df.dtypes

Agencia_ID                int32
Canal_ID                   int8
Cliente_ID                int32
Demanda_uni_equil       float64
Producto_ID               int32
Ruta_SAK                  int32
Semana                     int8
ZipCode                   int16
id                        int32
Last_per_Cliente_ID     float32
Last_per_Ruta_SAK       float32
week_ct                   int64
Log_Target_mean_lag1    float64
Log_Target_mean_lag2    float64
Log_Target_mean_lag3    float64
Log_Target_mean_lag4    float64
Lags_sum                float64
brand                     int64
Qty_Ruta_SAK_Bin          int64
num_prod                  int64
num_prod_uni              int64
dtype: object

In [33]:
df.to_csv('jorge_dataset_modified.csv',index=False) 